# Double Pendulum

In [2]:
# ] add OrdinaryDiffEqTsit5

   Resolving package versions...
  No Changes to `/workspaces/phys2200-lectures-jt20012/Project.toml`
  No Changes to `/workspaces/phys2200-lectures-jt20012/Manifest.toml`


In [3]:
# ] add Printf

   Resolving package versions...
    Updating `/workspaces/phys2200-lectures-jt20012/Project.toml`
  [de0858da] + Printf v1.11.0
  No Changes to `/workspaces/phys2200-lectures-jt20012/Manifest.toml`


In [4]:
# ] add Colors

   Resolving package versions...
    Updating `/workspaces/phys2200-lectures-jt20012/Project.toml`
  [5ae59095] + Colors v0.13.1
  No Changes to `/workspaces/phys2200-lectures-jt20012/Manifest.toml`


In [1]:
using PyPlot
using OrdinaryDiffEqTsit5
using Printf
using Colors

In [2]:

"""
            Function doublependulum!(dudt, u, p, t)

......
"""
function doublependulum!(dudt, u, p, t)
    # unpack variables and parameters
    theta1 = u[1]
    theta2 = u[2]
    dtheta1dt = u[3]
    dtheta2dt = u[4]
    κ, ν = p
    
    sn, cs = sincos(theta1 - theta2)
    a1 = κ*cs
    a2 = ν*cs
    γ = 1/(1.0 - a1*a2)
    
    f1 = -κ*sn*dtheta2dt^2 - sin(theta1)
    f2 =  ν*(sn*dtheta1dt^2 - ν*sin(theta2))

    dudt[1] = u[3]
    dudt[2] = u[4]
    dudt[3] = γ*(f1 - a1*f2)
    dudt[4] = γ*(f2 - a2*f1)
    
    return nothing
end


doublependulum!

In [3]:

l1 = 1.0
l2 = 1.0  
m1 = 1.0 
m2 = 1.0;

In [7]:
tspan = (0.0, 500.0)
initial = [pi, pi/100, 0.0, 0.0];

In [8]:

ν = l1/l2
κ = 1 / ν * m2 / (m1 + m2)
# Pack the parameters
p = (κ, ν);
prob = ODEProblem(doublependulum!, initial, tspan, p);

In [10]:
sol = solve(prob, Tsit5(), abstol=1e-7, reltol=1e-7);

In [11]:

np = 2001
t = range(tspan[1], tspan[2], np)
sl = sol(t);

In [12]:

theta1 = sl[1, :]
theta2 = sl[2, :]
x1 =  l1 .* sin.(theta1)
y1 = -l1 .* cos.(theta1)
x2 = x1 .+ l2 .* sin.(theta2)
y2 = y1 .- l2 .* cos.(theta2);

In [16]:
function snapshot_ft(nt, pts_disp, box, t, x1, y1, x2, y2)
    nt_start = max(1, nt-pts_disp)
    xl, xr, yb, yt = box
    axis("square")
    xlim(xl, xr)
    ylim(yb, yt)
    titl = @sprintf("Double pendulum t = %5.2f .. %5.2f",
                    t[nt_start], t[nt])
    title(titl, family="monospace")
    xlabel(L"$x$")
    ylabel(L"$y$")
    grid(true)

    # pivot
    plot(0.0, 0.0, color="black", marker="x", markersize=5.0)

    # two rods
    plot([0.0, x1[nt],x2[nt]], [0.0, y1[nt], y2[nt]], color="black")

    # mass 1
    c1 = "green"
    plot(x1[nt], y1[nt], color=c1, marker="o", markersize=5.0)
    
    # mass 2
    c2 = "blue"
    plot(x2[nt], y2[nt], color=c2, marker="o", markersize=5.0)

    # fading tail trajectory of mass 2
    c2p = parse(RGBA, c2)
    for i = nt_start:nt-1
        transp = 1.0 - ((nt-i-1)/(pts_disp-1))^2
        cc = (c2p.r, c2p.g, c2p.b, transp)
        plot([x2[i], x2[i+1]], [y2[i], y2[i+1]], color=cc, linewidth=0.5)
    end
    return nothing
end;

In [13]:
axis_lim = (l1 + l2)*1.2
xl = -axis_lim
xr = -xl
yb = -axis_lim
yt = -yb
box = (xl, xr, yb, yt);

In [14]:
f = 0.1                      # Fraction of displayed points
pts_disp = round(Int, f*np)  # Number of displayed points

200

In [ ]:

fig = figure()
for nt = 1:10:np
    snapshot_ft(nt, pts_disp, box, t, x1, y1, x2, y2)
    display(fig)
    IJulia.clear_output(true)
    clf()
end